In [1]:
import pandas as pd
from oaklib import get_adapter

In [2]:
RESULTS_DIR = "../results/manual"

In [3]:
from c3p.datamodel import Dataset

BENCH_DIR = "../results/2025/benchmark"
with open(f"{BENCH_DIR}/dataset.json", "r") as f:
    dataset = Dataset.model_validate_json(f.read())

In [4]:
name2smiles = {s.name: s.smiles for s in dataset.structures}

In [5]:
chebi = get_adapter("sqlite:obo:chebi")

In [6]:
# check the data...
BETALAMIC_ACID = "CHEBI:27483"
ALKALOID = "CHEBI:22315"


In [7]:
def get_smiles_from_chebi(chebi_id: str):
    return chebi.entity_metadata_map(chebi_id)["obo:chebi/smiles"][0]

BETALAMIC_ACID_SMILES = get_smiles_from_chebi(BETALAMIC_ACID)
BETALAMIC_ACID_SMILES

'OC(=O)[C@@H]1CC(=C/C=O)\\C=C(N1)C(O)=O'

In [8]:
alkaloid_cls = dataset.get_chemical_class_by_id(ALKALOID)
alkaloid_cls.name

'alkaloid'

In [9]:
BETALAMIC_ACID_SMILES in alkaloid_cls.all_positive_examples

True

In [10]:
curator_df = pd.read_csv(f"{RESULTS_DIR}/manual-evals.csv")

In [16]:
curator_df = curator_df[curator_df["chemical_class_id"].notna()]
curator_df

,chemical_class_id,chemical_class_name,smiles,fixed_smiles,structure_name,model=citeseek-gpt-4o_response,model=claude-sonnet_response,model=deepseek-chat_response,model=gpt-3.5-turbo_response,model=gpt-4o_response,Curator: is_member?,Curator: comments
0,CHEBI:20706,6-aminopurines,O=NN1CCNCC1,No change,N-Mononitrosopiperazine,"NO, the statement is incorrect. \n\nN-Mononitr...",NO. N-Mononitrosopiperazine (with SMILES 'O=NN...,NO. The structure N-Mononitrosopiperazine (wit...,NO\n\nExplanation: N-Mononitrosopiperazine is ...,"NO, the provided statement is incorrect. \n\nN...",No,it is a nitroso compound and a piperazine
1,CHEBI:20706,6-aminopurines,O1C(CCC1)C(=O)N2CCNCC2,No change,1-(Tetrahydro-2-furoyl)piperazine,"NO, the statement is not true.\n\n1-(Tetrahydr...",NO\n\nThe provided statement is false. The str...,NO. The structure 1-(Tetrahydro-2-furoyl)piper...,"NO\n\nExplanation: The structure provided, 1-(...","NO, 1-(Tetrahydro-2-furoyl)piperazine is not a...",No,it is a piperazine and a tetrahydrofuran
2,CHEBI:20706,6-aminopurines,NC1=NC=NC2=C1N=CN2[C@@H]3O[C@H](CO)[C@@H](OP(O...,No change,"5'-hydroxy-diadenylate with a cyclic 2',3' pho...","YES, the provided statement is true.\n\nThe st...","YES, the provided statement is true.\n\nExplan...","YES. The structure described, 5'-hydroxy-diade...",YES\n\nThe provided structure is indeed a type...,"YES, The provided structure '5'-hydroxy-diaden...",Yes,NaN
3,CHEBI:47622,acetate ester,CCCCCCCCCCCCCC(=O)OC[C@H](COP([O-])(=O)OC[C@@H...,CCCCCCCCCCCCCC(=O)OC[C@H](COP([O-])(=O)OC[C@@H...,1-myristoyl-2-oleoyl-sn-glycero-3-phosphatidyl...,"NO, the statement is not true. \n\nThe structu...","NO, the provided statement is not true.\n\nExp...",NO. The structure 1-myristoyl-2-oleoyl-sn-glyc...,NO\n\nExplanation: The provided SMILES string ...,"NO, the provided statement is incorrect. \n\n1...",No,It is a carboxylic ester
4,CHEBI:17984,acyl-CoA,CCC=C/CC=C/CC=C/CC=C/CC=C/CCCCCCCCCC(=O)SCCNC(...,CC\C=C/C\C=C/C\C=C/C\C=C/C\C=C/CCCCCCCCCC(=O)S...,"(11Z,14Z,17Z,20Z,23Z)-hexacosapentaenoyl-CoA(4-)","YES, the provided statement is true. \n\nThe S...","YES, the provided statement is true.\n\nThe st...","YES. The structure (11Z,14Z,17Z,20Z,23Z)-hexac...","YES, the statement is true. The provided compo...","YES, the statement is true. The structure desc...",No,It is an acyl CoA(4-). Its net charge is -4 wh...
...,...,...,...,...,...,...,...,...,...,...,...,...
217,CHEBI:61910,very long-chain fatty acyl-CoA,CCCCCCCCCCCCCCCCCCCCCCCCC(C(O)=O)C(=O)SCCNC(=O...,No change,2-carboxyhexacosanoyl-CoA,"YES, the provided statement is true. The struc...",YES\n\nThe provided SMILES string 'CCCCCCCCCCC...,YES. The structure 2-carboxyhexacosanoyl-CoA i...,YES\n\nExplanation: The provided SMILES struct...,"YES, the provided statement is true. The compo...",Yes,NaN
218,CHEBI:10036,wax ester,CCCCCCCCCCCCCCCC(=O)OCC(CCCCCCCCCCCCCC)OC(=O)C...,No change,2-palmitoyloxypalmityl palmitate,"YES, the provided statement is true. The given...","YES, the provided statement is true.\n\n2-palm...",YES. The structure 2-palmitoyloxypalmityl palm...,YES\n\nExplanation: The provided structure (2-...,YES. \n\nThe structure 2-palmitoyloxypalmityl ...,Yes,it is also a diester
219,CHEBI:10036,wax ester,CCCCCCCCCCOC(=O)CCCCCCCC=C/CCCCCCCC,CCCCCCCCCCOC(=O)CCCCCCC\C=C/CCCCCCCC,decyl oleate,"YES, the structure known as decyl oleate is a ...","YES, the provided statement is true.\n\nDecyl ...",YES. Decyl oleate is a wax ester. It is formed...,YES\n\nDecyl oleate is a type of wax ester. It...,"YES, the provided statement is true. Decyl ole...",Yes,NaN
220,CHEBI:10036,wax ester,OC(CCCCCCCCOC(=O)CCCCCCCCCCCCCCCCCCCCC)CCCC,No change,9-Hydroxytridecyl docosanoate,"YES, the statement is true. The structure 9-Hy...","YES, the provided statement is correct. 9-Hydr...",YES. The structure 9-Hydroxytridecyl docosanoa...,YES\n\nExplanation: The provided structure of ...,"YES, the statement is true. The structure know...",Yes,NaN


In [17]:
# fix SMILES
fixed = {}

def fix_smiles(name: str, curr_smiles: str):
    if name not in name2smiles:
        raise ValueError(f"Name not in dataset: {name}")
    fixed_smiles = name2smiles[name]
    if fixed_smiles != curr_smiles:
        fixed[name] = fixed_smiles
        # check if they differ other than backslashes
        if fixed_smiles.replace("\\", "") != curr_smiles.replace("\\", ""):
            raise ValueError(f"SMILES differ: {fixed_smiles} != {curr_smiles}")
    return fixed_smiles

In [18]:
curator_df["smiles"] = curator_df.apply(lambda row: fix_smiles(row["structure_name"], row["smiles"]), axis=1)

In [19]:
len(fixed)

50

In [20]:
# turned dict into a df
fixed_df = pd.DataFrame(list(fixed.items()), columns=["name", "smiles"])
fixed_df.to_csv(f"{RESULTS_DIR}/fixed-smiles.csv", index=False)
fixed_df.to_excel(f"{RESULTS_DIR}/fixed-smiles.xlsx", index=False)

In [21]:
CUR_IS_MEMBER = "Curator: is_member?"
CUR_COMMENTS = "Curator: comments"

In [22]:
curator_df.columns

Index(['chemical_class_id', 'chemical_class_name', 'smiles', 'fixed_smiles',
       'structure_name', 'model=citeseek-gpt-4o_response',
       'model=claude-sonnet_response', 'model=deepseek-chat_response',
       'model=gpt-3.5-turbo_response', 'model=gpt-4o_response',
       'Curator: is_member?', 'Curator: comments'],
      dtype='object')

In [38]:
curator_df.describe()

,chemical_class_id,chemical_class_name,smiles,fixed_smiles,structure_name,model=citeseek-gpt-4o_response,model=claude-sonnet_response,model=deepseek-chat_response,model=gpt-3.5-turbo_response,model=gpt-4o_response,Curator: is_member?,Curator: comments,in_benchmark
count,222,222,222,222,222,222,222,222,222,222,222,93,222
unique,94,94,201,51,201,222,222,222,222,222,3,81,2
top,CHEBI:61051,lipid hydroperoxide,CCCCCCCCCCCCCC(=O)OC[C@H](COP([O-])(=O)OC[C@@H...,No change,1-myristoyl-2-oleoyl-sn-glycero-3-phosphatidyl...,"NO, the statement is incorrect. \n\nN-Mononitr...",NO. N-Mononitrosopiperazine (with SMILES 'O=NN...,NO. The structure N-Mononitrosopiperazine (wit...,NO\n\nExplanation: N-Mononitrosopiperazine is ...,"NO, the provided statement is incorrect. \n\nN...",No,I may have said yes if i had ignored the charg...,False
freq,6,6,5,163,5,1,1,1,1,1,150,3,124


In [39]:
curator_df[curator_df[CUR_COMMENTS].notna()][["chemical_class_name", "structure_name", CUR_IS_MEMBER, CUR_COMMENTS]]

,chemical_class_name,structure_name,Curator: is_member?,Curator: comments
0,6-aminopurines,N-Mononitrosopiperazine,No,it is a nitroso compound and a piperazine
1,6-aminopurines,1-(Tetrahydro-2-furoyl)piperazine,No,it is a piperazine and a tetrahydrofuran
3,acetate ester,1-myristoyl-2-oleoyl-sn-glycero-3-phosphatidyl...,No,It is a carboxylic ester
4,acyl-CoA,"(11Z,14Z,17Z,20Z,23Z)-hexacosapentaenoyl-CoA(4-)",No,It is an acyl CoA(4-). Its net charge is -4 wh...
5,acyl-CoA,N-Mononitrosopiperazine,No,It is a piperazine
...,...,...,...,...
200,secondary alcohol,"1,2-bis(4-hydroxyphenyl)propan-2-ol",No,It seems like a tertiary alcohol (more precise...
202,steroid,"(5E,7E,10E)-(3S)-19-methyl-9,10-seco-5,7,10(19...",Yes,It is a secosteroid (a subclass of steroids)
203,steroid,"(20S,24R)-dihydroxyvitamin D3",Yes,It is a secosteroid (a subclass of steroids)
213,triterpenoid saponin,Diacetylpyxinol,No,structure is missing the glycoside group


In [34]:
def col_summary(col):
    return curator_df.groupby(col).size()

In [24]:
curator_df[CUR_IS_MEMBER].unique()

array(['No', 'Yes', 'Unknown'], dtype=object)

In [25]:
curator_df[curator_df[CUR_IS_MEMBER] == "Unknown"]

,chemical_class_id,chemical_class_name,smiles,fixed_smiles,structure_name,model=citeseek-gpt-4o_response,model=claude-sonnet_response,model=deepseek-chat_response,model=gpt-3.5-turbo_response,model=gpt-4o_response,Curator: is_member?,Curator: comments
83,CHEBI:72544,flavonoids,C12=C(C3=C(C(=CC(O3)=O)C4=CC=CC=C4)C5=C1C=CC(O...,No change,inophyllolide chromanol,"NO, the structure known as inophyllolide chrom...",NO. The provided structure of inophyllolide ch...,NO. The structure of inophyllolide chromanol (...,NO\n\nExplanation: Inophyllolide chromanol is ...,"NO, the statement is not true. \n\nInophylloli...",Unknown,could potentially be a flavonoid derivative
124,CHEBI:35381,monosaccharide,[H][C@@]1(OC(=O)C(O)=C1O)[C@@H](O)CO,No change,L-ascorbic acid,"NO, the provided statement is not true.\n\nL-a...",NO. The structure of L-ascorbic acid (vitamin ...,NO. L-ascorbic acid (vitamin C) is not a monos...,NO\n\nExplanation: L-ascorbic acid is not cons...,"NO, L-ascorbic acid is not a monosaccharide. \...",Unknown,There is some confusion in the literature rega...
126,CHEBI:25409,monoterpenoid,OC(=O)C[C@@H](CC/C=C(\C)/C(O)=O)C,OC(=O)C[C@@H](CC/C=C(\C)/C(O)=O)C,"2,6R-Dimethyl-2E-octen-1,8-dioic acid","NO, the statement is incorrect. \n\nThe struct...","NO, the provided statement is not true.\n\nExp...","NO. The structure 2,6R-Dimethyl-2E-octen-1,8-d...",NO\n\nExplanation: The provided structure is n...,"NO, the provided statement is not true. The st...",Unknown,I am unable to decide by looking at its chemic...


In [26]:
curator_df.aggregate({CUR_IS_MEMBER: ["count"]})

,Curator: is_member?
count,222


In [27]:
def classifies_in_benchmark(row: dict):
    cc = dataset.get_chemical_class_by_id(row["chemical_class_id"])
    return row["smiles"] in cc.all_positive_examples

curator_df["in_benchmark"] = curator_df.apply(classifies_in_benchmark, axis=1)
    

In [28]:
curator_df[curator_df["smiles"] == BETALAMIC_ACID_SMILES]

,chemical_class_id,chemical_class_name,smiles,fixed_smiles,structure_name,model=citeseek-gpt-4o_response,model=claude-sonnet_response,model=deepseek-chat_response,model=gpt-3.5-turbo_response,model=gpt-4o_response,Curator: is_member?,Curator: comments,in_benchmark
15,CHEBI:22315,alkaloid,OC(=O)[C@@H]1CC(=C/C=O)\C=C(N1)C(O)=O,OC(=O)[C@@H]1CC(=C/C=O)\C=C(N1)C(O)=O,betalamic acid,"NO, the provided statement is not true. \n\nBe...",NO. Betalamic acid (SMILES: OC(=O)[C@@H]1CC(=C...,NO. Betalamic acid is not an alkaloid. While i...,NO\n\nBetalamic acid is not typically consider...,"NO, betalamic acid is not classified as an alk...",No,is a dicarboxylic acid and an aldehyde.,True


In [29]:
betalamic_acid_smiles_retrieved = curator_df[curator_df["structure_name"] == "betalamic acid"]["smiles"].unique()[0]
betalamic_acid_smiles_retrieved

'OC(=O)[C@@H]1CC(=C/C=O)\\C=C(N1)C(O)=O'

In [30]:
betalamic_acid_smiles_retrieved, BETALAMIC_ACID_SMILES, betalamic_acid_smiles_retrieved == BETALAMIC_ACID_SMILES

('OC(=O)[C@@H]1CC(=C/C=O)\\C=C(N1)C(O)=O',
 'OC(=O)[C@@H]1CC(=C/C=O)\\C=C(N1)C(O)=O',
 True)

In [31]:
assert betalamic_acid_smiles_retrieved == BETALAMIC_ACID_SMILES

In [32]:
print(betalamic_acid_smiles_retrieved, BETALAMIC_ACID_SMILES)

OC(=O)[C@@H]1CC(=C/C=O)\C=C(N1)C(O)=O OC(=O)[C@@H]1CC(=C/C=O)\C=C(N1)C(O)=O


In [35]:
col_summary("in_benchmark")

in_benchmark
False    124
True      98
dtype: int64

In [41]:
from oaklib.datamodels.vocabulary import IS_A


def classifies_in_chebi(row: dict):
    inst_ids = chebi.curies_by_label(row["structure_name"])
    if len(inst_ids) != 1:
        raise ValueError(f"Ambiguous {row} ==> {inst_ids}")
    inst_id = inst_ids[0]
    return row["chemical_class_id"] in list(chebi.ancestors(inst_id, [IS_A]))

In [42]:
curator_df["in_chebi"] = curator_df.apply(classifies_in_chebi, axis=1)


In [43]:
col_summary("in_chebi")

in_chebi
False    124
True      98
dtype: int64

In [44]:
curator_df["agreement_benchmark_chebi"] = curator_df.apply(lambda row: row["in_chebi"] == row["in_benchmark"], axis=1)


In [45]:
def agrees(yn, v):
    if yn == "Unknown":
        return None
    return (yn == "Yes") == v
    
curator_df["agreement_curator_benchmark"] = curator_df.apply(lambda row: agrees(row[CUR_IS_MEMBER], row["in_benchmark"]), axis=1)
curator_df["agreement_curator_chebi"] = curator_df.apply(lambda row: agrees(row[CUR_IS_MEMBER], row["in_chebi"]), axis=1)


In [46]:
curator_df.groupby(["agreement_curator_benchmark"]).size()

agreement_curator_benchmark
False     94
True     125
dtype: int64

In [47]:
curator_df.groupby(["agreement_curator_chebi"]).size()


agreement_curator_chebi
False     94
True     125
dtype: int64

In [48]:
# get counts of each combination
curator_df.groupby([CUR_IS_MEMBER, "in_benchmark"]).size()

Curator: is_member?  in_benchmark
No                   False           90
                     True            60
Unknown              True             3
Yes                  False           34
                     True            35
dtype: int64

In [49]:
curator_df.groupby(["agreement_curator_benchmark"]).size()


agreement_curator_benchmark
False     94
True     125
dtype: int64

In [50]:
%load_ext autoreload
%autoreload 2

In [51]:
from pathlib import Path
from c3p.classifier import classify

smiles_strs = list(curator_df["smiles"].unique())
chemical_class_names = list(curator_df["chemical_class_name"].unique())


In [52]:
len(smiles_strs), len(chemical_class_names), len(smiles_strs) * len(chemical_class_names)

(201, 94, 18894)

In [53]:
results = list(classify(smiles_strs, program_directory=Path("../results/2025/ensembl-5/programs"), chemical_classes=chemical_class_names, strict=True))

In [54]:
len(results)

18894

In [55]:
results_by_pair = {(r.class_name, r.input_smiles): r.is_match for r in results}

In [56]:
#list(results_by_pair.keys())[0:10]

In [57]:
curator_df["in_c3p"] = curator_df.apply(lambda row: results_by_pair[row["chemical_class_name"], row["smiles"]], axis=1)


In [58]:
curator_df.groupby(["in_c3p"]).size()

in_c3p
False     98
True     124
dtype: int64

In [59]:
curator_df["agreement_c3p_curator"] = curator_df.apply(lambda row: agrees(row[CUR_IS_MEMBER], row["in_c3p"]), axis=1)


In [60]:
#curator_df["c3p_correct"]

In [61]:
curator_df.groupby(["agreement_c3p_curator"]).size()

agreement_c3p_curator
False    125
True      94
dtype: int64

In [62]:
curator_df.groupby(["agreement_c3p_curator"]).size() / len(curator_df)


agreement_c3p_curator
False    0.563063
True     0.423423
dtype: float64

In [63]:
curator_df_filtered = curator_df[curator_df[CUR_IS_MEMBER] != "Unknown"]

In [64]:
curator_df_filtered.groupby(["agreement_c3p_curator"]).size() / len(curator_df_filtered)

agreement_c3p_curator
False    0.570776
True     0.429224
dtype: float64

In [65]:
curator_df.groupby([CUR_IS_MEMBER, "in_benchmark", "in_c3p", "in_chebi"]).size()


Curator: is_member?  in_benchmark  in_c3p  in_chebi
No                   False         True    False       90
                     True          False   True        60
Unknown              True          False   True         3
Yes                  False         True    False       34
                     True          False   True        35
dtype: int64

In [66]:
curator_df[(curator_df["in_benchmark"] == False) & (curator_df["in_c3p"] == False)] 

,chemical_class_id,chemical_class_name,smiles,fixed_smiles,structure_name,model=citeseek-gpt-4o_response,model=claude-sonnet_response,model=deepseek-chat_response,model=gpt-3.5-turbo_response,model=gpt-4o_response,Curator: is_member?,Curator: comments,in_benchmark,in_chebi,agreement_benchmark_chebi,agreement_curator_benchmark,agreement_curator_chebi,in_c3p,agreement_c3p_curator


In [67]:
curator_df[curator_df["agreement_benchmark_chebi"] == False] 

,chemical_class_id,chemical_class_name,smiles,fixed_smiles,structure_name,model=citeseek-gpt-4o_response,model=claude-sonnet_response,model=deepseek-chat_response,model=gpt-3.5-turbo_response,model=gpt-4o_response,Curator: is_member?,Curator: comments,in_benchmark,in_chebi,agreement_benchmark_chebi,agreement_curator_benchmark,agreement_curator_chebi,in_c3p,agreement_c3p_curator


In [68]:
curator_df.to_csv(f"{RESULTS_DIR}/manual-evals-with-c3p.csv", index=False)

In [69]:
curator_df.to_excel(f"{RESULTS_DIR}/manual-evals-with-c3p.xlsx", index=False)